In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize simulator
sim = AerSimulator()

def run_and_print(qc, description):
    """Helper to run, simulate, and print results"""
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=1024).result()
    counts = result.get_counts()
    print(f"\n--- {description} ---")
    print(qc.draw())
    print("Measurement Counts:", counts)

# ===============================================================
# 1. ORIGINAL CIRCUIT
# ===============================================================
qc_original = QuantumCircuit(2, 2)
qc_original.h(0)        # Hadamard on qubit 0
qc_original.cx(0, 1)    # CNOT with control=0, target=1
qc_original.cz(0, 1)    # CZ with control=0, target=1
qc_original.swap(0, 1)  # Swap qubit 0 and 1
qc_original.measure_all()
run_and_print(qc_original, "Original Circuit")

# ===============================================================
# 2. CONTROL/TARGET VARIATION
# ===============================================================
qc_swapped_control = QuantumCircuit(2, 2)
qc_swapped_control.h(0)
qc_swapped_control.cx(1, 0)  # Control and target swapped
qc_swapped_control.cz(1, 0)  # Control and target swapped
qc_swapped_control.swap(0, 1)
qc_swapped_control.measure_all()
run_and_print(qc_swapped_control, "Control/Target Swapped in CNOT and CZ")

print("\nExplanation:")
print("Swapping control and target changes which qubit influences the other. "
      "For example, in CNOT, the control determines when the target flips. "
      "This changes the entanglement pattern and final measurement probabilities.")

# ===============================================================
# 3. GATE REMOVAL (Remove SWAP gate)
# ===============================================================
qc_no_swap = QuantumCircuit(2, 2)
qc_no_swap.h(0)
qc_no_swap.cx(0, 1)
qc_no_swap.cz(0, 1)
# SWAP removed
qc_no_swap.measure_all()
run_and_print(qc_no_swap, "Circuit Without SWAP Gate")

print("\nObservation:")
print("Removing the SWAP gate means the qubits retain their original positions, "
      "so measurement results correspond directly to qubits 0 and 1 without being exchanged.")

# ===============================================================
# 4. ADD ANOTHER HADAMARD ON SECOND QUBIT BEFORE CNOT
# ===============================================================
qc_extra_h = QuantumCircuit(2, 2)
qc_extra_h.h(0)
qc_extra_h.h(1)       # Added Hadamard on qubit 1
qc_extra_h.cx(0, 1)
qc_extra_h.cz(0, 1)
qc_extra_h.swap(0, 1)
qc_extra_h.measure_all()
run_and_print(qc_extra_h, "Added Hadamard on Qubit 1 Before CNOT")

print("\nExplanation:")
print("Adding a Hadamard on qubit 1 places it into superposition before entanglement. "
      "This changes interference effects and can lead to a more uniform or altered distribution "
      "of measurement outcomes across basis states.")



--- Original Circuit ---
        ┌───┐            ░ ┌─┐   
   q_0: ┤ H ├──■───■──X──░─┤M├───
        └───┘┌─┴─┐ │  │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──X──░──╫─┤M├
             └───┘       ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts: {'00 00': 511, '11 00': 513}

--- Control/Target Swapped in CNOT and CZ ---
        ┌───┐┌───┐       ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─■──X──░─┤M├───
        └───┘└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ───────■───■──X──░──╫─┤M├
                         ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts: {'00 00': 499, '10 00': 525}

Explanation:
Swapping control and target changes which qubit influences the other. For example, in CNOT, the control determines when the target flips. This changes the entanglement pattern and final measurement proba